In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import json
import os
from tkcalendar import Calendar  # Import Calendar widget
from datetime import datetime

class TaskManager:
    def __init__(self):
        self.tasks = []
        self.load_tasks()

    def load_tasks(self):
        """Load tasks from a file if it exists."""
        if os.path.exists('tasks.json'):
            with open('tasks.json', 'r') as file:
                self.tasks = json.load(file)

    def save_tasks(self):
        """Save the current tasks to a file."""
        with open('tasks.json', 'w') as file:
            json.dump(self.tasks, file)

    def add_task(self, task_name, category, due_date, priority):
        """Add a new task to the list."""
        task = {
            'name': task_name,
            'complete': False,
            'category': category,
            'due_date': due_date,
            'priority': priority
        }
        self.tasks.append(task)
        self.save_tasks()

    def edit_task(self, task_index, new_name, category, due_date, priority):
        """Edit an existing task's details."""
        if 0 <= task_index < len(self.tasks):
            self.tasks[task_index]['name'] = new_name
            self.tasks[task_index]['category'] = category
            self.tasks[task_index]['due_date'] = due_date
            self.tasks[task_index]['priority'] = priority
            self.save_tasks()
            return True
        return False

    def delete_task(self, task_index):
        """Delete a task from the list."""
        if 0 <= task_index < len(self.tasks):
            self.tasks.pop(task_index)
            self.save_tasks()
            return True
        return False

    def mark_task_complete(self, task_index):
        """Mark a task as complete."""
        if 0 <= task_index < len(self.tasks):
            self.tasks[task_index]['complete'] = True
            self.save_tasks()
            return True
        return False

    def get_tasks(self):
        """Return the current list of tasks."""
        return self.tasks

class App:
    def __init__(self, root):
        self.task_manager = TaskManager()
        self.root = root
        self.root.title("Task Manager")

        # Frame for the task list
        self.frame = tk.Frame(self.root)
        self.frame.pack(pady=10)

        self.task_listbox = tk.Listbox(self.frame, width=70, height=15)
        self.task_listbox.pack(side=tk.LEFT)

        self.scrollbar = tk.Scrollbar(self.frame)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.task_listbox.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.task_listbox.yview)

        # Entry for new tasks
        self.task_name_entry = tk.Entry(self.root, width=52)
        self.task_name_entry.pack(pady=10)

        # Combobox for categories
        self.category_var = tk.StringVar(self.root)
        self.category_combobox = ttk.Combobox(self.root, textvariable=self.category_var, values=["Personal", "Work", "School", "Fitness", "Other"], state="readonly")
        self.category_combobox.set("Select Category")  # Default value
        self.category_combobox.pack(pady=10)

        # Calendar for due date selection
        self.due_date_button = tk.Button(self.root, text="Select Due Date", command=self.open_calendar)
        self.due_date_button.pack(pady=10)
        self.due_date = None  # To store the selected date

        # Combobox for priority
        self.priority_var = tk.StringVar(self.root)
        self.priority_combobox = ttk.Combobox(self.root, textvariable=self.priority_var, values=["1 - High", "2 - Medium", "3 - Low"], state="readonly")
        self.priority_combobox.set("Select Priority")  # Default value
        self.priority_combobox.pack(pady=10)

        # Buttons
        self.add_task_button = tk.Button(self.root, text="Add Task", command=self.add_task)
        self.add_task_button.pack(pady=5)

        self.edit_task_button = tk.Button(self.root, text="Edit Task", command=self.edit_task)
        self.edit_task_button.pack(pady=5)

        self.delete_task_button = tk.Button(self.root, text="Delete Task", command=self.delete_task)
        self.delete_task_button.pack(pady=5)

        self.mark_complete_button = tk.Button(self.root, text="Mark Task Complete", command=self.mark_task_complete)
        self.mark_complete_button.pack(pady=5)

        self.populate_tasks()

    def open_calendar(self):
        """Open a calendar popup to select the due date."""
        top = tk.Toplevel(self.root)
        top.title("Select Due Date")
        
        self.calendar = Calendar(top, selectmode='day', year=datetime.now().year, month=datetime.now().month, day=datetime.now().day)
        self.calendar.pack(pady=20)
        
        select_button = tk.Button(top, text="Select", command=lambda: self.set_due_date(top))
        select_button.pack(pady=10)

    def set_due_date(self, calendar_window):
        """Set the selected date as the due date and close the calendar window."""
        self.due_date = self.calendar.get_date()
        calendar_window.destroy()

    def populate_tasks(self):
        """Populate the listbox with tasks from the task manager."""
        self.task_listbox.delete(0, tk.END)
        for task in self.task_manager.get_tasks():
            status = "✔️ Complete" if task['complete'] else "❌ Incomplete"
            display_text = f"{task['name']} - {status} | Due: {task['due_date']} | Category: {task['category']} | Priority: {task['priority']}"
            self.task_listbox.insert(tk.END, display_text)

    def add_task(self):
        """Add a new task from the entry fields."""
        task_name = self.task_name_entry.get()
        category = self.category_var.get()
        due_date = self.due_date
        priority = self.priority_var.get()

        # Check if task name, category, due date, and priority are valid
        if not task_name or category == "Select Category" or not due_date or priority == "Select Priority":
            messagebox.showwarning("Input Error", "Please fill in all fields.")
            return

        # Add the task with the selected details
        priority_value = int(priority[0])  # Get the numeric priority
        self.task_manager.add_task(task_name, category, due_date, priority_value)
        self.populate_tasks()
        self.clear_entries()

    def edit_task(self):
        """Edit the selected task."""
        try:
            selected_index = self.task_listbox.curselection()[0]
            new_name = self.task_name_entry.get()
            category = self.category_var.get()
            due_date = self.due_date
            priority = self.priority_var.get()
            
            if not due_date:
                messagebox.showwarning("Warning", "Please select a due date.")
                return
            
            if self.task_manager.edit_task(selected_index, new_name, category, due_date, int(priority[0])):
                self.populate_tasks()
                self.clear_entries()
            else:
                messagebox.showerror("Error", "Invalid task index.")
        except IndexError:
            messagebox.showwarning("Warning", "You must select a task to edit.")

    def delete_task(self):
        """Delete the selected task."""
        try:
            selected_index = self.task_listbox.curselection()[0]
            if self.task_manager.delete_task(selected_index):
                self.populate_tasks()
                self.clear_entries()
            else:
                messagebox.showerror("Error", "Invalid task index.")
        except IndexError:
            messagebox.showwarning("Warning", "You must select a task to delete.")

    def mark_task_complete(self):
        """Mark the selected task as complete."""
        try:
            selected_index = self.task_listbox.curselection()[0]
            if self.task_manager.mark_task_complete(selected_index):
                self.populate_tasks()
            else:
                messagebox.showerror("Error", "Invalid task index.")
        except IndexError:
            messagebox.showwarning("Warning", "You must select a task to mark as complete.")

    def clear_entries(self):
        """Clear the input fields."""
        self.task_name_entry.delete(0, tk.END)
        self.category_combobox.set("Select Category")
        self.due_date = None
        self.priority_combobox.set("Select Priority")

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()
